In [1]:
!pip install recommenders
!pip install cornac



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 9.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 25.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.7/750.7 kB 19.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 40.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 63.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
train_data = pd.read_csv("/kaggle/input/my-data/train.csv")
test_data = pd.read_csv("/kaggle/input/my-data/test.csv")
print(train_data.head())
print(test_data.head())

   userID           Tag  rating  itemID
0       1     algorithm      37      40
1       1        arrays     497      12
2       1          hash       6     160
3       1     hashtable       1     165
4       1  optimization       7     112
   userID                Tag  rating  itemID
0       4  memory-management       4     177
1       4          recursion       3     185
2       4             arrays       3      12
3       4         heuristics       6     213
4       4       optimization       5     112


In [3]:
# Rename the columns
train_data.rename(columns={'OwnerUserId': 'userID', 'TagIndex': 'itemID', 'Score': 'rating'}, inplace=True)
test_data.rename(columns={'OwnerUserId': 'userID', 'TagIndex': 'itemID', 'Score': 'rating'}, inplace=True)


In [4]:
import os
import sys
import cornac
import pandas as pd

from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.utils.notebook_utils import store_metadata

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

System version: 3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
Cornac version: 1.18


In [5]:
# top k items to recommend
TOP_K = 5

# Model parameters
NUM_FACTORS = 25
NUM_EPOCHS = 100

In [6]:
train_data = train_data[["userID", "itemID", "rating"]]
train_data = train_data.sample(frac=1.0, random_state=42)

In [7]:
train_set = cornac.data.Dataset.from_uir(train_data.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 220095
Number of items: 228


In [8]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [9]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/100 [00:00<?, ?it/s]

Optimization finished!
Took 32.5832 seconds for training.


In [10]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train_data, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 124.6661 seconds for prediction.


In [11]:

all_predictions.head(10)

,userID,itemID,prediction
0,1,0,1.408445
1,1,1,-1.866581
2,1,2,-1.169506
3,1,3,-1.599172
4,1,4,-2.801036
5,1,5,-3.910197
6,1,6,-2.502327
7,1,7,-2.933540
8,1,8,-2.997454
9,1,9,-0.579282


In [13]:
k = 5
eval_map = map(test_data, all_predictions, col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test_data, all_predictions, col_prediction='prediction', k=k)
eval_precision = precision_at_k(test_data, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test_data, all_predictions, col_prediction='prediction', k=k)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

/opt/conda/lib/python3.10/site-packages/recommenders/evaluation/python_evaluation.py:438: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  df_hit.groupby(col_user, as_index=False)[col_user].agg({"hit": "count"}),
/opt/conda/lib/python3.10/site-packages/recommenders/evaluation/python_evaluation.py:439: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  rating_true_common.groupby(col_user, as_index=False)[col_user].agg(
/opt/conda/lib/python3.10/site-packages/recommenders/evaluation/python_evaluation.py:438: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  df_hit.groupby(col_user, as_index=False)[col_user].agg({"hit": "count"}),
/opt/conda/lib/python3.10/site-packages/recomm

MAP:	0.435804
NDCG:	0.575710
Precision@K:	0.505710
Recall@K:	0.505710


/opt/conda/lib/python3.10/site-packages/recommenders/evaluation/python_evaluation.py:438: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  df_hit.groupby(col_user, as_index=False)[col_user].agg({"hit": "count"}),
/opt/conda/lib/python3.10/site-packages/recommenders/evaluation/python_evaluation.py:439: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  rating_true_common.groupby(col_user, as_index=False)[col_user].agg(


In [14]:
tags_list = [
    "boolean", "boolean-operations", "boolean-logic", "boolean-expression", "boolean-algebra", "boolean-polynomials",
    "proof", "induction", "proof-of-correctness",
    "probability", "probability-theory","abstract-data-type","arrays", "static-array", "array-comparison", "array-reverse", "array-address", "array-pointer","dynamic-arrays","dynamic-allocation", "array-merge",
    "sparse-array", "arrayofarrays", "multidimensional-array", "sparse-matrix", "matrix-multiplication","array-multisort", "variable-length-array",
    "linked-list", "list", "singly-linked-list", "doubly-linked-list","string", "string-operations", "bitstring", "substring", "string-concatenation", "string-substitution",
    "string-comparison", "string-matching","algorithm","array-algorithms","quicksort", "mergesort", "radix-sort", "linear-search", "bucket-sort", "ternary-search",
    "counting-sort", "lexicographic-ordering", "combinatorics", "binary-search", "bubble-sort", "selection-sort",
    "sorting", "insertion-sort", "stable-sort","data-structures","stack", "queue","queueing",
    "enqueue","heap","heapalloc","heap-memory","heap-table","heapsort","binary-heap","fibonacci-heap", "priority-queue", "circular-queue", "fifo", "stack-memory", "binomial-heap", "lifo",
    "stack-allocation", "min-heap", "max-heap","minmax-heap", "heap-size","heaps-algorithm","tree", "treenode", "treepath", "tree-search", "tree-balancing", "tree-rotation", "tree-traversal",
    "breadth-first-search", "depth-first-search", "best-first-search", "treesort", "binary-tree", "ternary-tree",
    "multiway-tree", "binary-search-tree", "balanced-binary-search-tree", "ternary-search-tree", "avl-tree", "b-tree",
    "b-plus-tree", "red-black-tree", "red-black-tree-insertion", "splay-tree", "hashtree", "n-ary-tree", "huffman-tree",
    "huffman-code","string-algorithm", "longest-substring", "lcs", "greedy", "optimization", "mathematical-optimization",
    "linear-programming","knuth-morris-pratt", "rabin-karp","graph", "graph-theory", "strongly-connected-graph", "adjacency-list", "adjacency-matrix", "directed-graph",
    "undirected-graph", "digraphs", "subgraph", "minimum-spanning-tree", "minimum-spanning-forest",
    "directed-acyclic-graphs", "graph-traversal", "a-star", "Dijkstra", "kruskals-algorithm", "prims-algorithm",
    "hungarian-algorithm", "simplex", "simplex-algorithm", "strassen", "clique", "clique-problem",
    "graph-algorithm", "tarjans-algorithm", "shortest-path", "planar-graph", "floyd-warshall",
    "traveling-salesman", "path-finding", "bipartite", "hamiltonian-cycle", "topological-sort", "graph-coloring",
    "spanning-tree", "bellman-ford", "kosaraju-algorithm", "euler-path", "graph-query", "dependency-graph",
    "vertex-cover", "hamiltonian-path", "cyclic-graph","hash", "hashmap", "hash-function", "hashset", "double-hashing", "hashtable", "hashcode", "hash-collision",
    "consistent-hashing","dynamic-programming", "knapsack-problem", "code-complexity", "big-o", "tail-recursion", "partitioning",
    "time-complexity","space-complexity","memory-management", "network-flow", "branch-and-bound", "max-flow", "master-theorem", "recursive-backtracking",
    "fibonacci", "towers-of-hanoi", "recursion", "divide-and-conquer", "asymptotic-complexity",
    "greatest-common-divisor", "partition-problem", "ford-fulkerson", "edmonds-karp", "set", "set-cover",
    "amortized-analysis", "set-intersection", "set-union", "set-theory", "set-difference", "set-operations","set-cover"
    "disjoint-union", "disjoint-sets", "union-find","complexity-theory", "recursive-datastructures", "p-np", "np", "np-hard", "np-complete", "sat",
    "satisfiability", "2-satisfiability", "stable-marriage", "heuristics", "artificial-intelligence", "automata",
    "automata-theory", "finite-automata", "non-deterministic", "subset-sum", "minimax", "turing-machines",
    "turing-complete", "halting-problem", "backtracking", "formal-languages", "stability", "computability",
    "probing", "linear-probing", "quadratic-probing"]

len(set(tags_list))


231

In [15]:
import pandas as pd
import numpy as np

# Function to get known tags for a user based on their ratings
def get_known_tags(user_id, ratings_df, tags_list):
    # Filter the ratings dataframe for the specified user and positive ratings
    user_ratings = ratings_df[(ratings_df['userID'] == user_id) & (ratings_df['rating'] > 0)]
    
    if user_ratings.empty:
        print(f"No positive ratings found for user {user_id}")
    
    # Get the unique itemIDs (which correspond to tag indices)
    tag_indices = user_ratings['itemID'].unique().astype(int)  # Convert to integer
    
    # Map the tag indices to tag names
    known_tags = [tags_list[i] for i in tag_indices]
    
    return known_tags

# Function to get top 10 predictions for a user excluding known tags
def get_top_10_predictions(user_id, predictions_df, tags_list, known_tags):
    # Filter the predictions dataframe for the specified user
    user_predictions = predictions_df[predictions_df['userID'] == user_id]
    
    # Sort the predictions by 'prediction' column in descending order
    sorted_predictions = user_predictions.sort_values(by='prediction', ascending=False)
    
    # Map the itemIDs (which correspond to tag indices) to tag names and exclude known tags
    top_10_tags = []
    for _, row in sorted_predictions.iterrows():
        tag = tags_list[int(row['itemID'])]
        if tag not in known_tags:
            top_10_tags.append((tag, row['prediction']))
        if len(top_10_tags) == 10:
            break
    
    return top_10_tags

# Main function to recommend tags for a user
def recommend_tags(user_id, ratings_df, predictions_df, tags_list):
    # Get known tags
    known_tags = get_known_tags(user_id, ratings_df, tags_list)
    
    # Get top 10 predictions excluding known tags
    top_10_tags = get_top_10_predictions(user_id, predictions_df, tags_list, known_tags)
    
    return known_tags, top_10_tags

# Example usage
# Assuming train_data and all_predictions are already loaded DataFrames
# tags_list = [...]  # Provide the list of tags here

# Function to get a random user with positive ratings
def get_random_user_id(ratings_df):
    users_with_positive_ratings = ratings_df[ratings_df['rating'] > 0]['userID'].unique()
    return np.random.choice(users_with_positive_ratings)

# Running the recommendation for a different user each time
np.random.seed()  # Remove the fixed seed to get a different user each run
random_user_id = get_random_user_id(train_data)

known_tags, top_10_tags = recommend_tags(random_user_id, train_data, all_predictions, tags_list)

print(f"Random UserID: {random_user_id}")
print("Known Tags for User:", known_tags)
print("Top 10 Predicted Tags for User:")
for rank, (tag, prediction) in enumerate(top_10_tags, start=1):
    print(f"{rank}: {tag} (Prediction: {prediction})")


Random UserID: 3273107
Known Tags for User: ['list', 'hashmap']
Top 10 Predicted Tags for User:
1: algorithm (Prediction: 2.903451442718506)
2: recursion (Prediction: 2.3798887729644775)
3: multidimensional-array (Prediction: 2.270352363586426)
4: optimization (Prediction: 2.1437363624572754)
5: data-structures (Prediction: 2.005352020263672)
6: hash (Prediction: 1.7658661603927612)
7: memory-management (Prediction: 1.684338927268982)
8: set (Prediction: 1.4760000705718994)
9: substring (Prediction: 1.4457863569259644)
10: linked-list (Prediction: 1.2454301118850708)


In [45]:
import pandas as pd
import numpy as np

# Function to get known tags for a user based on their ratings
def get_known_tags(user_id, ratings_df, tags_list):
    # Filter the ratings dataframe for the specified user and positive ratings
    user_ratings = ratings_df[(ratings_df['userID'] == user_id) & (ratings_df['rating'] > 0)]
    
    if user_ratings.empty:
        print(f"No positive ratings found for user {user_id}")
    
    # Get the unique itemIDs (which correspond to tag indices)
    tag_indices = user_ratings['itemID'].unique().astype(int)  # Convert to integer
    
    # Map the tag indices to tag names
    known_tags = [tags_list[i] for i in tag_indices]
    
    return known_tags

# Function to get top 10 predictions for a user excluding known tags
def get_top_10_predictions(user_id, predictions_df, tags_list, known_tags):
    # Filter the predictions dataframe for the specified user
    user_predictions = predictions_df[predictions_df['userID'] == user_id]
    
    # Sort the predictions by 'prediction' column in descending order
    sorted_predictions = user_predictions.sort_values(by='prediction', ascending=False)
    
    # Map the itemIDs (which correspond to tag indices) to tag names and exclude known tags
    top_10_tags = []
    for _, row in sorted_predictions.iterrows():
        tag = tags_list[int(row['itemID'])]
        if tag not in known_tags:
            top_10_tags.append((tag, row['prediction']))
        if len(top_10_tags) == 10:
            break
    
    return top_10_tags

# Function to get the test set for a user
def get_user_test_set(user_id, test_df):
    user_test_set = test_df[test_df['userID'] == user_id]
    if user_test_set.empty:
        return None
    return user_test_set

# Main function to recommend tags for a user
def recommend_tags(user_id, ratings_df, predictions_df, tags_list, test_df):
    # Get known tags
    known_tags = get_known_tags(user_id, ratings_df, tags_list)
    
    # Get top 10 predictions excluding known tags
    top_10_tags = get_top_10_predictions(user_id, predictions_df, tags_list, known_tags)
    
    # Get the test set for the user
    user_test_set = get_user_test_set(user_id, test_df)
    
    return known_tags, top_10_tags, user_test_set

# Example usage
# Assuming train_data, all_predictions, and test_data are already loaded DataFrames
# tags_list = [...]  # Provide the list of tags here

# Function to get a random user with positive ratings
def get_random_user_id(ratings_df):
    users_with_positive_ratings = ratings_df[ratings_df['rating'] > 0]['userID'].unique()
    return np.random.choice(users_with_positive_ratings)

# Read the test CSV file
test_data = pd.read_csv('/kaggle/input/my-data/test.csv')
train_data = pd.read_csv('/kaggle/input/my-data/train.csv')

# Running the recommendation for a different user each time
np.random.seed()  # Remove the fixed seed to get a different user each run
random_user_id = get_random_user_id(train_data)

known_tags, top_10_tags, user_test_set = recommend_tags(random_user_id, train_data, all_predictions, tags_list, test_data)
# Your list of recommendations (replace with your actual recommendations)
recommendations = []
print(f"Random UserID: {random_user_id}")
print("Known Tags for User:", known_tags)
print("Top 10 Predicted Tags for User:")
for rank, (tag, prediction) in enumerate(top_10_tags, start=1):
    print(f"{rank}: {tag} (Prediction: {prediction})")
    recommendations.append(tag)

if user_test_set is not None:
    print("\nUser Test Set:")
    print(user_test_set)
else:
    print("\nNo Test Set Available for User")

print(recommendations)

Random UserID: 364696
Known Tags for User: ['a-star', 'algorithm', 'arrays', 'big-o', 'binary-search', 'boolean', 'boolean-expression', 'boolean-logic', 'breadth-first-search', 'combinatorics', 'complexity-theory', 'data-structures', 'doubly-linked-list', 'fibonacci', 'graph', 'hash', 'hashtable', 'heap', 'heap-memory', 'heapsort', 'linked-list', 'list', 'memory-management', 'mergesort', 'multidimensional-array', 'optimization', 'priority-queue', 'probability', 'queue', 'quicksort', 'recursion', 'recursive-datastructures', 'set', 'set-difference', 'sorting', 'space-complexity', 'stable-sort', 'string', 'string-algorithm', 'string-comparison', 'string-matching', 'substring', 'time-complexity']
Top 10 Predicted Tags for User:
1: matrix-multiplication (Prediction: 1.3800346851348877)
2: stack (Prediction: 1.2922627925872803)
3: insertion-sort (Prediction: 1.215512990951538)
4: binary-tree (Prediction: 1.1537493467330933)
5: greatest-common-divisor (Prediction: 0.8571100234985352)
6: tree 

In [46]:
import networkx as nx

def read_graph(file_path):
    try:
        G = nx.read_gml(file_path)
        print(f"Graph loaded with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
        return G
    except Exception as e:
        print(f"An error occurred while reading the graph: {e}")
        return nx.DiGraph()

def topological_sort_graph(G):
    try:
        topological_order = list(nx.topological_sort(G))
        #print("Topological sort order:", topological_order)
        return topological_order
    except nx.NetworkXUnfeasible:
        print("The graph is not a Directed Acyclic Graph (DAG).")
        return []

def reorder_recommendations(topological_order, recommendations):
    # Filter the topological order to include only the tags in the recommendations
    recommendation_set = set(recommendations)
    ordered_recommendations = [tag for tag in topological_order if tag in recommendation_set]
    return ordered_recommendations

# Load the graph
graph_file_path = '/kaggle/input/pre-req2/maximal_spanning_tree-DN.graph'
G = read_graph(graph_file_path)

# Perform topological sort on the graph
topological_order = topological_sort_graph(G)

# Reorder the recommendations based on the topological sort order
ordered_recommendations = reorder_recommendations(topological_order, recommendations)
print("Recommendations:", recommendations)
print("Ordered Recommendations:", ordered_recommendations)


Graph loaded with 231 nodes and 230 edges.
Recommendations: ['matrix-multiplication', 'stack', 'insertion-sort', 'binary-tree', 'greatest-common-divisor', 'tree', 'radix-sort', 'huffman-code', 'hashmap', 'tail-recursion']
Ordered Recommendations: ['hashmap', 'matrix-multiplication', 'insertion-sort', 'radix-sort', 'huffman-code', 'greatest-common-divisor', 'tree', 'stack', 'tail-recursion', 'binary-tree']


In [ ]:
import pandas as pd
import cornac
from cornac.eval_methods import CrossValidation
from cornac.metrics import MAP, NDCG, Precision, Recall
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.timer import Timer

# Load and rename the data
train_data = pd.read_csv("/kaggle/input/my-data/train.csv")
test_data = pd.read_csv("/kaggle/input/my-data/test.csv")
train_data.rename(columns={'OwnerUserId': 'userID', 'TagIndex': 'itemID', 'Score': 'rating'}, inplace=True)
test_data.rename(columns={'OwnerUserId': 'userID', 'TagIndex': 'itemID', 'Score': 'rating'}, inplace=True)

# Prepare the train set
train_data = train_data[["userID", "itemID", "rating"]]

# Parameters
k_list = [5, 10, 25]
num_factors_list = [10, 25, 50, 100]
NUM_EPOCHS = 100
SEED = 42

best_eval = None
best_model = None
best_num_factors = None
best_k = None
best_all_predictions = None

for num_factors in num_factors_list:
    print(f"Evaluating BPR with num_factors={num_factors}")
    bpr = cornac.models.BPR(
        k=num_factors,
        max_iter=NUM_EPOCHS,
        learning_rate=0.01,
        lambda_reg=0.001,
        verbose=True,
        seed=SEED
    )

    # Train the model
    train_set = cornac.data.Dataset.from_uir(train_data.itertuples(index=False), seed=SEED)
    with Timer() as t:
        bpr.fit(train_set)
    print("Took {} seconds for training.".format(t))

    # Predict rankings
    with Timer() as t:
        all_predictions = predict_ranking(bpr, train_data, usercol='userID', itemcol='itemID', remove_seen=True)
    print("Took {} seconds for prediction.".format(t))

    # Evaluate the model for each k in k_list
    for k in k_list:
        eval_map = map(test_data, all_predictions, col_prediction='prediction', k=k)
        eval_ndcg = ndcg_at_k(test_data, all_predictions, col_prediction='prediction', k=k)
        eval_precision = precision_at_k(test_data, all_predictions, col_prediction='prediction', k=k)
        eval_recall = recall_at_k(test_data, all_predictions, col_prediction='prediction', k=k)

        print(f"Evaluation Metrics for num_factors={num_factors}, top_k={k}:")
        print("MAP:\t%f" % eval_map)
        print("NDCG:\t%f" % eval_ndcg)
        print("Precision@K:\t%f" % eval_precision)
        print("Recall@K:\t%f" % eval_recall)

        # Save the best metrics and corresponding predictions
        if best_eval is None or eval_map > best_eval:
            best_eval = eval_map
            best_model = bpr
            best_num_factors = num_factors
            best_k = k
            best_all_predictions = all_predictions

print(f"Best model found with num_factors={best_num_factors}, top_k={best_k}:")
print("MAP:\t%f" % best_eval)

# Final best predictions
best_all_predictions = predict_ranking(best_model, train_data, usercol='userID', itemcol='itemID', remove_seen=True)

# Save the best predictions as a variable
final_best_all_predictions = best_all_predictions

# Optionally save to a file
final_best_all_predictions.to_csv("best_all_predictions.csv", index=False)

print("Final best predictions saved.")


Synthetic Data

In [26]:
import pandas as pd
train_data = pd.read_csv("/kaggle/input/my-data/train.csv")
test_data = pd.read_csv("/kaggle/input/my-data/test.csv")
print(train_data.head())
print(test_data.head())

Random UserID: 4415950
Known Tags for User: ['algorithm', 'arrays', 'binary-search-tree', 'list']
Top 10 Predicted Tags for User:
1: string (Prediction: 3.4683477878570557)
2: sorting (Prediction: 2.7690494060516357)
3: data-structures (Prediction: 2.5492730140686035)
4: recursion (Prediction: 2.379197835922241)
5: tree (Prediction: 2.1079139709472656)
6: multidimensional-array (Prediction: 1.8374801874160767)
7: linked-list (Prediction: 1.8024604320526123)
8: optimization (Prediction: 1.6275368928909302)
9: memory-management (Prediction: 1.3425796031951904)
10: hash (Prediction: 1.0948541164398193)
